In [1674]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1675]:
import pandas as pd

from functions import *

pd.set_option('display.max_columns', None)

df_merged = pd.read_csv("data/df_merged_national.csv")

df_poblacion = pd.read_csv("data/cleaned_pop.csv")


In [1676]:
df_merged = clean_df(df_merged)
df_merged = every_float_to_int(df_merged)

In [1677]:
df_no_provincias = df_merged[(df_merged["region"] == "    DESCONOCIDA") | (df_merged["region"] == "    EN EL EXTRANJERO") | (df_merged["region"] == "    TOTAL NACIONAL")]

df_no_provincias.to_csv("data/df_no_provincias.csv", index=False)

In [1678]:
df_merged = df_merged[~df_merged.index.isin(df_no_provincias.index)]

In [1679]:
df_num = df_merged.iloc[:, df_merged.columns != 'region']

In [1680]:
df_merged = add_coordinates_from_dict(df_merged)

In [1681]:
df_merged["region"] = convert_region_into_numeric(df_merged["region"], True)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Project6_CrimesSpain\functions.py:228: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [1682]:
df_merged.insert(0, "year", df_merged.pop("year"))

In [1683]:
df_merged = delete_sub_crimes(df_merged)

In [1684]:
df_merged["avg_crimes"] = df_merged.iloc[:, 2:-3].mean(axis=1)      # [:, 2:-2] quiero -> todas las filas, desde la 2da hasta la penultima columna

In [1685]:
df_poblacion.drop(["Nacional", "Sexo"], inplace=True, axis=1)

df_poblacion["region"] = convert_region_into_numeric(df_poblacion["region"], False)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Project6_CrimesSpain\functions.py:228: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [1686]:
df_merged = df_merged.merge(df_poblacion, on=["region", "year"])

df_merged.rename(columns={"Total": "population"}, inplace=True)

In [1687]:
crime_values = ['1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration']

In [1688]:
df_unpivot = pd.melt(df_merged, id_vars=['year', 'region', 'latitude', 'longitude', 'population'], value_vars=crime_values)

In [1689]:
df_unpivot_transformed = df_unpivot.copy()

for crime in crime_values:
    column_name = f'{crime}_indicator'
    df_unpivot_transformed[column_name] = (df_unpivot_transformed['variable'] == crime).astype(int)


In [1690]:
df_values = get_previous_years(df_unpivot_transformed)


In [1691]:
df_total_values = df_values.dropna(subset=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']).reset_index(drop=True)

In [1692]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
columns_pca = ['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']
values_year_data = df_total_values[columns_pca]
scaler = StandardScaler()
values_year_scaled = scaler.fit_transform(values_year_data)
pca = PCA(n_components=0.9, random_state=42)
pca_result = pca.fit_transform(values_year_scaled)

df_total_values["value_year_PCA"] = pca_result
df_total_values.drop(columns=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3'], inplace=True)


df_total_values["population"] = df_total_values["population"].replace("\.", "", regex=True).astype(int)

df_total_values.drop(columns=["region"], inplace=True)

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Iván\AppData\Local\Temp\ipykernel_25816\3038364401.py:14: SyntaxWarning: invalid escape sequence '\.'
  df_total_values["population"] = df_total_values["population"].replace("\.", "", regex=True).astype(int)


In [1693]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 12));
# sns.heatmap(df_total_values_201017.corr(), annot=True, cmap='YlGnBu', linewidths=0.5)    

<Figure size 1500x1200 with 0 Axes>

# Obtención de X_train, X_test, y_train, y_test

In [1694]:
df_total_values_201017 = df_total_values[df_total_values['year'] <= 2017]

df_total_values_2018 = df_total_values[df_total_values['year'] == 2018]

df_total_values_2019 = df_total_values[df_total_values['year'] == 2019]

In [1695]:
X_train = df_total_values_201017.drop(columns=['values_year'])
y_train = df_total_values_201017['values_year']

X_test = df_total_values_2018.drop(columns=['values_year'])
y_test = df_total_values_2018['values_year']

In [1697]:
#2. NORMALIZACIÓN DE DATOS

from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm = normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [1698]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)        # Vuelvo a dar formato de df
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [1699]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

bagging_reg_norm = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 500)

bagging_reg_norm.fit(X_train_norm, y_train)         # Entreno mi modelo

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20), max_samples=500,
                 n_estimators=100)

In [1700]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

pred_norm = bagging_reg_norm.predict(X_test_norm)

print("MAE", mean_absolute_error(pred_norm, y_test))
print("RMSE", root_mean_squared_error(pred_norm, y_test))
print("R2 score", bagging_reg_norm.score(X_test_norm, y_test))

MAE 997.7874533556114
RMSE 4883.72315767954
R2 score 0.988882105410223


In [1701]:
# import joblib
# joblib.dump(normalizer, "normalizer.pkl")

In [1702]:
df_total_values_2018['predict_values_year'] = pred_norm

C:\Users\Iván\AppData\Local\Temp\ipykernel_25816\1403187491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_values_2018['predict_values_year'] = pred_norm


In [1704]:
df_2010_2018 = df_total_values.copy()

In [1705]:
df_2010_2018 = df_2010_2018[df_2010_2018["year"] <= 2018]

In [1706]:
df_2010_2018 = df_2010_2018.merge(df_total_values_2018[["predict_values_year"]],left_index=True, right_index=True, how='left')

In [1707]:
df_2010_2018

,year,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,value_year_PCA,predict_values_year
19,2018,37.3873,-5.9869,8384408,35043,1,0,0,0,0,0,0,0,0,1.018908,35438.98
20,2018,41.6488,-0.8891,1308728,4474,1,0,0,0,0,0,0,0,0,-0.302771,3627.90
21,2018,43.3619,-5.8494,1028244,3159,1,0,0,0,0,0,0,0,0,-0.331669,3129.34
22,2018,39.5712,2.6466,1128908,7545,1,0,0,0,0,0,0,0,0,-0.192667,6488.64
23,2018,28.2916,-16.6291,2127685,11805,1,0,0,0,0,0,0,0,0,0.022440,9792.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,42.6954,-1.6761,644477,198,0,0,0,0,0,0,0,0,1,-0.458865,NaN
1193,2013,43.2630,-2.9349,2191682,1470,0,0,0,0,0,0,0,0,1,-0.416703,NaN
1194,2013,42.2871,-2.5396,322027,115,0,0,0,0,0,0,0,0,1,-0.462375,NaN
1195,2013,35.8894,-5.3198,84180,125,0,0,0,0,0,0,0,0,1,-0.463435,NaN


In [1708]:
import numpy as np


df_2010_2018["values_year"] = df_2010_2018.apply(lambda x : x['predict_values_year'] if not pd.isna(x["predict_values_year"]) else x["values_year"], axis=1)

In [1709]:
df_2010_2018.drop(columns=['predict_values_year'], inplace=True)

In [1710]:
df_2010_2018

,year,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,value_year_PCA
19,2018,37.3873,-5.9869,8384408,35438.98,1,0,0,0,0,0,0,0,0,1.018908
20,2018,41.6488,-0.8891,1308728,3627.90,1,0,0,0,0,0,0,0,0,-0.302771
21,2018,43.3619,-5.8494,1028244,3129.34,1,0,0,0,0,0,0,0,0,-0.331669
22,2018,39.5712,2.6466,1128908,6488.64,1,0,0,0,0,0,0,0,0,-0.192667
23,2018,28.2916,-16.6291,2127685,9792.89,1,0,0,0,0,0,0,0,0,0.022440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,42.6954,-1.6761,644477,198.00,0,0,0,0,0,0,0,0,1,-0.458865
1193,2013,43.2630,-2.9349,2191682,1470.00,0,0,0,0,0,0,0,0,1,-0.416703
1194,2013,42.2871,-2.5396,322027,115.00,0,0,0,0,0,0,0,0,1,-0.462375
1195,2013,35.8894,-5.3198,84180,125.00,0,0,0,0,0,0,0,0,1,-0.463435


In [1711]:
future_years = [2018]

features_nn = [
    'year','latitude', 'longitude', 'population',
    '1_against_people_indicator', '2_against_freedom_indicator',
    '3_sexual_freedom_indicator', '4_family_relationships_indicator',
    '5_property_crimes_indicator', '6_collective_security_indicator',
    '7_forgeries_indicator', '8_public_administration_indicator',
    '9_justice_administration_indicator', 'value_year_PCA'
    ]

df_try_1 = df_total_values_201017.copy()

for year in future_years:
    X_year = df_try_1[df_try_1['year'] == year - 1][features_nn]

    y_pred_year = bagging_reg_norm.predict(X_year)

    new_year_df = df_try_1[df_try_1['year']==year-1].copy()
    new_year_df['year'] = year
    new_year_df['values_year'] = y_pred_year

    df_try_1 = pd.concat([df_try_1, new_year_df], ignore_index=True)

    y_year = df_total_values_201819[df_total_values_201819['year'] == year]['values_year']

    if not y_year.empty:
        mae = mean_absolute_error(y_year, y_pred_year)
        rmse = root_mean_squared_error(y_year, y_pred_year)
        r2 = r2_score(y_year, y_pred_year)

        print(f"{year}")
        print(f"Mean Absolute Error (MAE): {mae}")
        print(f"Root Mean Squared Error (RMSE): {rmse}")
        print(f"R² Score: {r2}")

2018
Mean Absolute Error (MAE): 23956.165204678353
Root Mean Squared Error (RMSE): 74884.29251367558
R² Score: -1.6139780700563144


# Obtención de predicción de 2018, haciendo que datos de 2017 sean los predecidos

In [1712]:
df_total_values_201018 = df_total_values_201017

In [1713]:
df_total_values_2018 = df_total_values[df_total_values['year'] == 2018]

X_train_201017 = df_total_values_201017.drop(columns=['values_year'])
X_test_2018 = df_total_values_2018.drop(columns=['values_year'])

y_train_201017 = df_total_values_201017['values_year']
y_test_2018 = df_total_values_2018['values_year']

In [1714]:
X_train_norm_201017 = normalizer.transform(X_train_201017)        

X_test_norm_18 = normalizer.transform(X_test_2018)

In [1715]:
X_train_norm_201017 = pd.DataFrame(X_train_norm_201017, columns = X_train.columns)        # Vuelvo a dar formato de df

X_test_2018 = pd.DataFrame(X_test_2018, columns = X_test.columns)

In [1716]:
bagging_reg_norm_2018 = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 500)

bagging_reg_norm_2018.fit(X_train_norm_201017, y_train_201017)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20), max_samples=500,
                 n_estimators=100)

In [1717]:
X_test_2018 = X_test_2018.drop(columns=['prediction', 'real'])

KeyError: "['prediction', 'real'] not found in axis"

In [521]:
pred_norm_2018 = bagging_reg_norm_2018.predict(X_test_2018)

print("MAE", mean_absolute_error(pred_norm_2018, y_test_2018))
print("RMSE", root_mean_squared_error(pred_norm_2018, y_test_2018))
print("R2 score", bagging_reg_norm.score(X_test_norm_18, y_test_2018))

MAE 23875.433567251468
RMSE 73168.27324729832
R2 score 0.9890191612185312


c:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingRegressor was fitted with feature names
  warnings.warn(


In [522]:
df_total_values_2018["predict"] = pred_norm_2018

df_total_values_2018["predict"] = pred_norm_2018

C:\Users\Iván\AppData\Local\Temp\ipykernel_25816\1531958611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_values_2018["predict"] = pred_norm_2018
C:\Users\Iván\AppData\Local\Temp\ipykernel_25816\1531958611.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_values_2018["predict"] = pred_norm_2018


In [523]:
df_total_values_2018

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,value_year_PCA,predict
19,2018,0,37.3873,-5.9869,8384408,35043,1,0,0,0,0,0,0,0,0,1.018908,339706.06
20,2018,1,41.6488,-0.8891,1308728,4474,1,0,0,0,0,0,0,0,0,-0.302771,11.61
21,2018,2,43.3619,-5.8494,1028244,3159,1,0,0,0,0,0,0,0,0,-0.331669,11.61
22,2018,3,39.5712,2.6466,1128908,7545,1,0,0,0,0,0,0,0,0,-0.192667,11.51
23,2018,5,28.2916,-16.6291,2127685,11805,1,0,0,0,0,0,0,0,0,0.022440,7498.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2018,16,42.6954,-1.6761,647554,407,0,0,0,0,0,0,0,0,1,-0.455145,11.61
1098,2018,17,43.2630,-2.9349,2199088,1502,0,0,0,0,0,0,0,0,1,-0.411588,11.61
1099,2018,18,42.2871,-2.5396,315675,185,0,0,0,0,0,0,0,0,1,-0.462450,11.61
1100,2018,9,35.8894,-5.3198,85144,178,0,0,0,0,0,0,0,0,1,-0.461895,11.61
